# Step 1: v4 피처 엔지니어링 및 데이터 생성

## 📌 목적
이 노트북은 v1 전처리 데이터를 기반으로 v4 피처를 생성하고, 학습/테스트 데이터를 준비합니다.

## 🔄 프로세스
1. **v1 데이터 로드**: 기본 전처리가 완료된 train/test 데이터
2. **v3.5 피처 생성**: Hybrid Top150 + 도메인 파생변수 6개
3. **v4 신규 피처 추가**: 희귀 세그먼트 특화 15개 피처
4. **최종 데이터 저장**: 167개 피처 포함 parquet 파일

## 📊 출력물
- `data/df_master_v4_train.parquet`: 학습 데이터 (Segment 포함)
- `data/df_master_v4_test.parquet`: 테스트 데이터 (Segment 없음)

---

## 🛠️ 코드 실행

아래 셀을 순차적으로 실행하면 v4 피처가 생성됩니다.

### 주요 함수 설명
- `yyyymmdd_to_datetime()`: 날짜 변환 함수
- `add_v35_features()`: v3.5 피처 생성 (Top150 + 도메인 6개)
- `add_v4_features()`: v4 신규 피처 15개 생성

---

In [ ]:
# step1_build_v4_features_train_test.py

import os
from pathlib import Path
import numpy as np
import pandas as pd

# --------------------------------
# 0) 경로 설정
# --------------------------------
# 프로젝트 루트: final_submission 기준
PROJECT_ROOT = Path(r"C:\Users\User\전산통계프로젝트\final_submission")
DATA_DIR     = PROJECT_ROOT / "data"
FEAT_DIR     = DATA_DIR / "features"

# 원본 데이터 경로 (v1 전처리 데이터 위치)
ORIGINAL_DATA_DIR = Path(r"C:\Users\User\전산통계프로젝트\data")

print("PROJECT_ROOT :", PROJECT_ROOT)
print("DATA_DIR     :", DATA_DIR)
print("FEAT_DIR     :", FEAT_DIR)
print("ORIGINAL_DATA_DIR :", ORIGINAL_DATA_DIR)

EPS   = 1e-6
TODAY = pd.Timestamp("2024-12-31")   # 기준일 (예전 코드와 동일)

# --------------------------------
# 1) v3.5 / v4 FE 함수 정의
#    (예전에 쓰던 add_v35_features, add_v4_features 그대로)
# --------------------------------

def yyyymmdd_to_datetime(s: pd.Series) -> pd.Series:
    """
    YYYYMMDD 형태 숫자/문자 시리즈를 datetime으로 변환.
    잘못된 값은 NaT 처리.
    """
    s_num = pd.to_numeric(s, errors="coerce").astype("Int64")
    return pd.to_datetime(s_num, format="%Y%m%d", errors="coerce")


def add_v35_features(df: pd.DataFrame, feat_dir: Path) -> tuple[pd.DataFrame, list]:
    """
    v3.5 Hybrid Top150 + 도메인 FE 6개를 df에 추가하고,
    최종 v3.5 feature 이름 리스트를 함께 리턴.
    """
    df_fe = df.copy()

    # 1) Hybrid Top150 로드
    top150_path = feat_dir / "top150_final.parquet"
    top150_final = pd.read_parquet(top150_path)

    base_feats_150 = top150_final["feature"].tolist()
    print("Loaded top150_final:", top150_final.shape)
    print("[Hybrid Top150 Feature 개수]:", len(base_feats_150))

    # 2) v3.5 도메인 FE 6개 생성
    # (1) 최근 3개월 오프라인 비율
    df_fe["v3_offline_ratio_R3M"] = df_fe["이용금액_오프라인_R3M"] / (
        df_fe["이용금액_오프라인_R3M"] + df_fe["이용금액_R3M_신용체크"] + EPS
    )

    # (2) 12M 일시불 중 고액 한 건 비율
    df_fe["v3_big_spend_ratio_R12M"] = df_fe["최대이용금액_일시불_R12M"] / (
        df_fe["이용금액_일시불_R12M"] + EPS
    )

    # (3) 청구금액 R3M vs R6M 변화율
    df_fe["v3_bill_change_R3M_R6M"] = (
        df_fe["청구금액_R3M"] - df_fe["청구금액_R6M"]
    ) / (df_fe["청구금액_R6M"] + EPS)

    # (4) B5/B2/B0 평균 청구금액
    df_fe["v3_bill_mean_B5_B2_B0"] = df_fe[
        ["정상청구원금_B5M", "정상청구원금_B2M", "정상청구원금_B0M"]
    ].mean(axis=1)

    # (5) B5 대비 B0 청구금액 변화율
    df_fe["v3_bill_change_B0_B5"] = (
        df_fe["정상청구원금_B0M"] - df_fe["정상청구원금_B5M"]
    ) / (df_fe["정상청구원금_B5M"] + EPS)

    # (6) 신용 이용 강도 (이용금액대 × log(1+신용건수))
    credit_cnt = df_fe["이용건수_신용_R12M"].clip(lower=0)
    df_fe["v3_credit_intensity"] = df_fe["이용금액대"] * np.log1p(credit_cnt)

    feats_v35_custom = [
        "v3_offline_ratio_R3M",
        "v3_big_spend_ratio_R12M",
        "v3_bill_change_R3M_R6M",
        "v3_bill_mean_B5_B2_B0",
        "v3_bill_change_B0_B5",
        "v3_credit_intensity",
    ]

    # 최종 v3.5 feature 리스트
    v35_features = base_feats_150 + feats_v35_custom
    v35_features = list(dict.fromkeys(v35_features))  # 중복 제거 + 순서 유지

    print("\n[v3.5 커스텀 파생변수 6개]:", feats_v35_custom)
    print("[v3.5 최종 Feature 개수]:", len(v35_features))

    # 실제 df 안에 없는 컬럼 체크 (안전장치)
    missing_cols = [c for c in v35_features if c not in df_fe.columns]
    print("\n[v3.5] df에 없는 feature 수:", len(missing_cols))
    if missing_cols:
        print("누락 컬럼 예시:", missing_cols[:20])
    else:
        print("✅ v3.5 feature 전부 df에 존재")

    return df_fe, v35_features


def add_v4_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    v3.5까지 FE가 들어있는 df에
    v4 희귀 세그 전용 파생변수를 추가해서 리턴.
    """
    df = df.copy()

    # =============================
    # 1) 날짜 컬럼 datetime 변환
    # =============================
    date_cols = [
        "최종이용일자_CA",
        "최종이용일자_일시불",
        "최종이용일자_신판",
        "최종이용일자_할부",
        "최종이용일자_기본",
        "최종이용일자_카드론",
    ]

    for col in date_cols:
        if col in df.columns:
            df[col + "_dt"] = yyyymmdd_to_datetime(df[col])
        else:
            df[col + "_dt"] = pd.NaT  # 없으면 NaT

    # ① v4_last_use_gap_CA
    df["v4_last_use_gap_CA"] = (TODAY - df["최종이용일자_CA_dt"]).dt.days

    # ② v4_last_use_gap_card_all
    last_use_all = df[[
        "최종이용일자_일시불_dt",
        "최종이용일자_신판_dt",
        "최종이용일자_할부_dt",
        "최종이용일자_기본_dt",
    ]].max(axis=1)
    df["v4_last_use_gap_card_all"] = (TODAY - last_use_all).dt.days

    # ③ v4_first_to_last_gap (가입 ~ 최종 이용까지 기간)
    if "rv최초시작후경과일" in df.columns:
        rv_days = pd.to_numeric(df["rv최초시작후경과일"], errors="coerce")

        max_reasonable_days = 365 * 200
        rv_days = rv_days.mask(rv_days > max_reasonable_days, np.nan)

        rv_delta = pd.to_timedelta(rv_days, unit="D")
        join_date = TODAY - rv_delta

        df["v4_first_to_last_gap"] = (
            df["최종이용일자_기본_dt"] - join_date
        ).dt.days
    else:
        df["v4_first_to_last_gap"] = np.nan

    # ④ v4_limit_to_usage_ratio_R12M
    num_12m = (
        df.get("이용금액_일시불_R12M", 0)
        + df.get("이용금액_할부_R12M", 0)
        + df.get("이용금액_CA_R12M", 0)
    )
    limit_amt = df.get("카드이용한도금액", 0)
    df["v4_limit_to_usage_ratio_R12M"] = num_12m / (limit_amt + EPS)

    # ⑤ v4_balance_to_usage_ratio
    num_bal = df.get("평잔_6M", 0)
    den_use_6m = (
        df.get("이용금액_일시불_R6M", 0)
        + df.get("이용금액_할부_R6M", 0)
        + df.get("이용금액_CA_R6M", 0)
        + EPS
    )
    df["v4_balance_to_usage_ratio"] = num_bal / den_use_6m

    # ⑥ v4_bill_drop_R6_to_R3
    bill_6 = df.get("청구금액_R6M", 0)
    bill_3 = df.get("청구금액_R3M", 0)
    df["v4_bill_drop_R6_to_R3"] = (bill_6 - bill_3) / (bill_6 + EPS)

    # ⑦ v4_usage_volatility_R3_R6_R12
    use_matrix = pd.concat(
        [
            df.get("이용금액_일시불_R3M",  pd.Series(0, index=df.index)),
            df.get("이용금액_일시불_R6M",  pd.Series(0, index=df.index)),
            df.get("이용금액_일시불_R12M", pd.Series(0, index=df.index)),
        ],
        axis=1,
    )
    df["v4_usage_volatility_R3_R6_R12"] = use_matrix.std(axis=1)

    # ⑧ v4_recent_zero_usage_flag
    recent_3m_sum = (
        df.get("이용금액_일시불_R3M", 0)
        + df.get("이용금액_할부_R3M", 0)
        + df.get("이용금액_CA_R3M", 0)
    )
    df["v4_recent_zero_usage_flag"] = (recent_3m_sum == 0).astype("int8")

    # ⑨ v4_long_inactive_high_limit_flag
    use_12m = (
        df.get("이용금액_일시불_R12M", 0).fillna(0) +
        df.get("이용금액_할부_R12M", 0).fillna(0) +
        df.get("이용금액_CA_R12M", 0).fillna(0)
    )
    last_basic_gap = (TODAY - df["최종이용일자_기본_dt"]).dt.days

    cond_high_limit    = df.get("카드이용한도금액", 0) > 3_000_000
    cond_low_usage     = use_12m < 100_000
    cond_long_inactive = last_basic_gap > 365

    df["v4_long_inactive_high_limit_flag"] = (
        cond_high_limit & cond_low_usage & cond_long_inactive
    ).astype("int8")

    # ⑩ v4_point_activity_intensity
    point_sum = (
        df.get("포인트_적립포인트_R12M", 0)
        + df.get("포인트_이용포인트_R12M", 0)
    )
    spend_12m = (
        df.get("이용금액_일시불_R12M", 0)
        + df.get("이용금액_할부_R12M", 0)
        + EPS
    )
    df["v4_point_activity_intensity"] = point_sum / spend_12m

    # ⑪ v4_travel_mileage_activity
    mileage_sum = (
        df.get("마일_적립포인트_R12M", 0)
        + df.get("마일_이용포인트_R12M", 0)
    )
    spend_12m_only = df.get("이용금액_일시불_R12M", 0) + EPS
    df["v4_travel_mileage_activity"] = mileage_sum / spend_12m_only

    # ⑫ v4_lifestyle_auto_payment_flag
    comm = df.get("납부_통신비이용금액", 0)
    fuel = df.get("교통_주유이용금액", 0)
    df["v4_lifestyle_auto_payment_flag"] = (
        (comm == 0) & (fuel == 0)
    ).astype("int8")

    # ⑬ v4_arrears_recent_flag
    arrears_recent = df.get("연체일수_최근", 0)
    df["v4_arrears_recent_flag"] = (
        arrears_recent > 30
    ).astype("int8")

    # ⑭ v4_cardloan_cleanup_flag
    loan_used       = df.get("카드론이용금액_누적", 0)
    loan_balance_b0 = df.get("잔액_카드론_B0M", 0)
    loan_last_gap   = (TODAY - df["최종이용일자_카드론_dt"]).dt.days

    cond_used            = loan_used > 1_000_000
    cond_zero_balance    = loan_balance_b0 == 0
    cond_long_since_loan = loan_last_gap > 365

    df["v4_cardloan_cleanup_flag"] = (
        cond_used & cond_zero_balance & cond_long_since_loan
    ).astype("int8")

    # ⑮ v4_online_offline_usage_ratio_R6M
    online_6m  = df.get("이용금액_온라인_R6M", 0)
    offline_6m = df.get("이용금액_오프라인_R6M", 0)
    df["v4_online_offline_usage_ratio_R6M"] = (
        online_6m / (online_6m + offline_6m + EPS)
    )

    # 날짜 도움 컬럼 정리
    drop_cols = [c for c in df.columns if c.endswith("_dt")]
    df.drop(columns=drop_cols, inplace=True)

    return df


# --------------------------------
# 2) train/test v4 만들기
# --------------------------------
if __name__ == "__main__":
    # 1) v1 train/test 로드 (원본 데이터 디렉토리에서)
    train_v1_path = ORIGINAL_DATA_DIR / "preprocessed" / "df_master_preprocessed_v1_train.parquet"
    test_v1_path  = ORIGINAL_DATA_DIR / "preprocessed" / "df_master_preprocessed_v1_test.parquet"

    train_v1 = pd.read_parquet(train_v1_path)
    test_v1  = pd.read_parquet(test_v1_path)

    print("\n[Loaded v1 train]", train_v1.shape)
    print("[Loaded v1 test ]", test_v1.shape)

    # 2) v3.5 + v4 FE (train)
    print("\n=== v3.5 + v4 FE (train) ===")
    train_v35, v35_features = add_v35_features(train_v1, FEAT_DIR)
    train_v4 = add_v4_features(train_v35)
    print("[train_v4 shape]:", train_v4.shape)

    # 3) v3.5 + v4 FE (test)
    print("\n=== v3.5 + v4 FE (test) ===")
    test_v35, _ = add_v35_features(test_v1, FEAT_DIR)
    test_v4 = add_v4_features(test_v35)
    print("[test_v4 shape]:", test_v4.shape)

    # 4) v4_feature_list.csv 기준으로 컬럼 정렬/선택 (train 기준)
    v4_feat_path = FEAT_DIR / "v4_feature_list.csv"
    if v4_feat_path.exists():
        v4_list = pd.read_csv(v4_feat_path)
        v4_feature_names = v4_list["feature"].tolist()
        print("\n[v4_feature_list 개수]:", len(v4_feature_names))
    else:
        raise FileNotFoundError(f"{v4_feat_path}가 없습니다. features 폴더 확인 필요")

    # --- train: Segment + v4_feature_names ---
    if "Segment" not in train_v4.columns:
        raise KeyError("train_v4에 Segment 컬럼이 없습니다. v1 전처리에서 Segment가 드랍되지 않았는지 확인 필요.")

    final_cols_train = ["Segment"] + [f for f in v4_feature_names if f in train_v4.columns]
    missing_train = [f for f in v4_feature_names if f not in train_v4.columns]

    print("\n[train] 최종 사용 feature 수:", len(final_cols_train)-1, "(+Segment)")
    print("[train] v4_list 기준 누락 feature 수:", len(missing_train))
    if missing_train:
        print("누락 예시:", missing_train[:20])

    df_master_v4_train = train_v4[final_cols_train].copy()
    print("[df_master_v4_train shape]:", df_master_v4_train.shape)

    # --- test: Segment 없이, train과 동일 feature만 사용 ---
    final_cols_test = [f for f in v4_feature_names if f in test_v4.columns]
    missing_test = [f for f in v4_feature_names if f not in test_v4.columns]

    print("\n[test] 최종 사용 feature 수:", len(final_cols_test))
    print("[test] v4_list 기준 누락 feature 수:", len(missing_test))
    if missing_test:
        print("누락 예시:", missing_test[:20])

    df_master_v4_test = test_v4[final_cols_test].copy()
    print("[df_master_v4_test shape]:", df_master_v4_test.shape)

    # 5) 저장 (final_submission/data 폴더에)
    out_train = DATA_DIR / "df_master_v4_train.parquet"
    out_test  = DATA_DIR / "df_master_v4_test.parquet"

    df_master_v4_train.to_parquet(out_train)
    df_master_v4_test.to_parquet(out_test)

    print("\n✅ Saved v4:")
    print(" -", out_train)
    print(" -", out_test)


NEW_PROJECT_ROOT : C:\Users\User\전산통계프로젝트
DATA_DIR_NEW     : C:\Users\User\전산통계프로젝트\data
OLD_PROJECT_ROOT : C:\Users\User\credit_segmentation-v4
FEAT_DIR_OLD     : C:\Users\User\credit_segmentation-v4\features

[Loaded v1 train] (400000, 851)
[Loaded v1 test ] (100000, 850)

=== v3.5 + v4 FE (train) ===
Loaded top150_final: (150, 4)
[Hybrid Top150 Feature 개수]: 150

[v3.5 커스텀 파생변수 6개]: ['v3_offline_ratio_R3M', 'v3_big_spend_ratio_R12M', 'v3_bill_change_R3M_R6M', 'v3_bill_mean_B5_B2_B0', 'v3_bill_change_B0_B5', 'v3_credit_intensity']
[v3.5 최종 Feature 개수]: 156

[v3.5] df에 없는 feature 수: 0
✅ v3.5 feature 전부 df에 존재
[train_v4 shape]: (400000, 872)

=== v3.5 + v4 FE (test) ===
Loaded top150_final: (150, 4)
[Hybrid Top150 Feature 개수]: 150

[v3.5 커스텀 파생변수 6개]: ['v3_offline_ratio_R3M', 'v3_big_spend_ratio_R12M', 'v3_bill_change_R3M_R6M', 'v3_bill_mean_B5_B2_B0', 'v3_bill_change_B0_B5', 'v3_credit_intensity']
[v3.5 최종 Feature 개수]: 156

[v3.5] df에 없는 feature 수: 0
✅ v3.5 feature 전부 df에 존재
[test_v4 s